In [2]:
# !pip install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from flask import Flask, request

app = Flask(__name__)

@app.route('/webhook', methods=['POST'])
def webhook():
    data = request.json
    # 處理數據...
    return 'OK', 200

if __name__ == '__main__':
    app.run(port=5000)


In [3]:
# !pip install --upgrade openai --quiet


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import openai
print(openai.__version__)

1.35.4


In [1]:
import openai
import os
import configparser

# 設定 OpenAI API 金鑰
# 讀取 config.ini 中的配置
config = configparser.ConfigParser()
config.read('config.ini')

# 從配置文件中獲取 OpenAI API 金鑰
OPENAI_API_KEY = config['DEFAULT']['OPENAI_API_KEY']
openai.api_key = OPENAI_API_KEY

def generate_response(user_message):
    try:
        # completion = client.chat.completions.create(
        #     model=MODEL,
        #     messages=[
        #     {"role": "system", "content": "You are a helpful assistant. Help me with my math homework!"}, # <-- This is the system message that provides context to the model
        #     {"role": "user", "content": "Hello! Could you solve 2+2?"}  # <-- This is the user message for which the model will generate a response
        #   ]
        # )
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": user_message}
            ],
            max_tokens=150
        )
        reply_message = response.choices[0].message.content
        # completion.choices[0].message.content
        return reply_message
    except Exception as e:
        return f"An error occurred: {str(e)}"

if __name__ == "__main__":
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            break
        response = generate_response(user_input)
        print(f"Assistant: {response}")


You:  你好


Assistant: 你好！有什么我可以帮助你的吗？


You:  今天天氣如何


Assistant: 抱歉，我無法提供即時的天氣資訊。不過，你可以通過手機上的天氣應用程式或是訪問天氣預報網站來查詢今天的天氣情況。如果你需要我幫助你使用這些應用程式或網站，請讓我知道。


You:  幫我查查今天的新聞


Assistant: 抱歉，我無法查詢即時新聞。但是，你可以訪問熱門新聞網站如《中央社》、《聯合報》、《蘋果日報》以及其他新聞網站來獲取最新的資訊。如有其他問題或需要幫助，我隨時為你效勞！


KeyboardInterrupt: Interrupted by user

# 正確調用4o 用法

In [7]:
from openai import OpenAI 
import os
import configparser

config = configparser.ConfigParser()
config.read('config.ini')

# 從配置文件中獲取 OpenAI API 金鑰
OPENAI_API_KEY = config['DEFAULT']['OPENAI_API_KEY']
client.api_key = OPENAI_API_KEY
## Set the API key and model name
MODEL="gpt-4o"
# client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as an env var>"))

completion = client.chat.completions.create(
  model=MODEL,
  messages=[
    {"role": "system", "content": "You are a helpful assistant. Help me with my math homework!"}, # <-- This is the system message that provides context to the model
    {"role": "user", "content": "Hello! Could you solve 2+2?"}  # <-- This is the user message for which the model will generate a response
  ]
)

print("Assistant: " + completion.choices[0].message.content)

Assistant: Of course! The sum of 2 + 2 is 4. If you have any more questions or need further assistance with your math homework, feel free to ask!


# Assistant API 測試

In [18]:
# !pip install PyMuPDF
# ! pip install openai PyPDF2

In [16]:
import fitz  # PyMuPDF
import openai
import configparser

# 設定 OpenAI API 金鑰
config = configparser.ConfigParser()
config.read('config.ini')

# 從配置文件中獲取 OpenAI API 金鑰
OPENAI_API_KEY = config['DEFAULT']['OPENAI_API_KEY']
openai.api_key = OPENAI_API_KEY

def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

pdf_text = extract_text_from_pdf("data/基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf")

with open("temp.txt", "w", encoding="utf-8") as temp_file:
    temp_file.write(pdf_text)

# 使用新版 API 客戶端
client = openai.OpenAI(api_key=OPENAI_API_KEY)

# 上傳文件
file_response = client.files.create(
    file=open("temp.txt", "rb"),
    purpose='assistants'
)
file_id = file_response['id']  # 使用字典訪問屬性

# 創建助手並附加文件
assistant_response = client.assistants.create(
    name="PDF_RAG_Assistant",
    model="gpt-4-turbo",
    tools=[{"type": "file_search"}],
    tool_resources={
        "file_search": {
            "file_ids": [file_id]
        }
    }
)
assistant_id = assistant_response['id']  # 使用字典訪問屬性

def search_relevant_sentences(query, assistant_id, top_k=5):
    search_response = client.assistants.search(
        assistant_id=assistant_id,
        query=query,
        max_rerank=top_k
    )
    return [item['text'] for item in search_response['data']]  # 使用字典訪問屬性

query = "用戶提出的問題"
relevant_sentences = search_relevant_sentences(query, assistant_id, top_k=5)

def generate_answer(query, context):
    prompt = f"使用以下信息來回答問題：\n{context}\n\n問題：{query}\n回答："
    completion_response = client.chat_completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150
    )
    return completion_response['choices'][0]['message']['content']

context = "\n".join(relevant_sentences)
answer = generate_answer(query, context)
print(answer)


TypeError: 'FileObject' object is not subscriptable

In [26]:
import openai
import configparser
from PyPDF2 import PdfReader

# Set up OpenAI API key
config = configparser.ConfigParser()
config.read('config.ini')
OPENAI_API_KEY = config['DEFAULT']['OPENAI_API_KEY']
client = openai.OpenAI(api_key=OPENAI_API_KEY)

def read_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

def query_openai(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",  # or another appropriate model
        messages=[
            {"role": "system", "content": "You are a helpful assistant that answers questions based on provided PDF content."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

# Read PDF text
pdf_text = read_pdf("data/基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf")

# # Construct query prompt
# user_question = "你的问题"
# query_prompt = f"以下是PDF文档中的内容：\n{pdf_text}\n根据这些内容回答以下问题：{user_question}"

# # Get answer
# answer = query_openai(query_prompt)
# print("回答：", answer)

In [27]:
# Construct query prompt
user_question = "這份文件在說什麼?"
query_prompt = f"以下是PDF文档中的内容：\n{pdf_text}\n根据这些内容回答以下问题：{user_question}"

# Get answer
answer = query_openai(query_prompt)
print("回答：", answer)

回答： 这份文件主要在解释癌症的基本概念、成因、命名方法以及不同的治疗方案。这是一份内部使用的图解文件，旨在通过图示和文字帮助读者理解以下几个方面：

1. **什么是癌症** - 详细解释癌症的起因，涉及细胞、DNA、基因突变等基础知识。
2. **癌症的成因** - 讨论突变如何发生，包括外在因子（例如吸烟、饮酒、化学物质暴露、感染等）和遗传因素。
3. **癌症的命名** - 说明癌症通常根据原发肿瘤的位置命名（如乳癌、肝癌、肺癌等）。
4. **常见治疗方法** - 介绍传统的治疗方法如化学治疗，手术和放射线治疗。
5. **现代治疗方法** - 讨论更先进的治疗策略，如标靶治疗、免疫治疗等。
6. **DNA突变检测** - 解释如何通过基因组检测识别相关的DNA突变，提升治疗的精确性。
7. **全方位癌症基因检测（CGP）** - 介绍CGP如何能够提供更广泛和详细的基因信息，帮助制定个性化的治疗方案。
8. **癌症的治疗决策** - 解释如何利用基因检测和其他信息帮助医生制定更精准的治疗计划。

文件还引用了多篇学术文献和资源来支持其内容，并且强调每位癌症患者都是独特的，需要个性化的治疗方案。文件目的是提升读者对癌症的理解及其治疗手段的认知，以便做出更科学和有效的治疗决策。


In [28]:
# Construct query prompt
user_question = "請幫我整理每一頁的內容"
query_prompt = f"以下是PDF文档中的内容：\n{pdf_text}\n根据这些内容回答以下问题：{user_question}"

# Get answer
answer = query_openai(query_prompt)
print("回答：", answer)

回答： 好的，以下是每一页内容的整理摘要：

1. **什么是癌症？**
   - 解释人体器官、组织和细胞的构成，以及细胞核、基因体、染色体、DNA和基因的相互关系。
   
2. **什么是癌症？**
   - 描述细胞生命周期的有限性和细胞通过生长和分裂进行更新，同时DNA在分裂过程中的复制。
   
3. **什么是癌症？**
   - 说明DNA复制时可能发生错误，这些错误被称为“突变”。

4. **什么是癌症？**
   - 解释突变可能由外在因子（如：抽烟、喝酒、化学/辐射物质、细菌或病毒感染）或遗传（先天遗传变异）引起。

5. **什么是癌症？**
   - 描述突变影响细胞正常运作和生长，最终导致细胞失控，形成恶性肿瘤并可能发生癌症转移。

6. **癌症的命名**
   - 说明癌症通常根据原发肿瘤的位置命名，如乳癌、肝癌、肺癌等。

7. **化学治疗**
   - 描述传统的癌症疗法，例如化学治疗，并指出随着对癌症DNA突变的了解，治疗方案可能因病人的不同而异。

8. **每位癌症病人都是独特的**
   - 强调不同病人可能有不同的DNA突变，因此个人化治疗变得重要。

9. **每位癌症病人都是独特的**
   - 进一步说明不同病人可能有不同的突变基因和数目，举例说明肺癌可能有许多种突变基因谱。

10. **精准医疗**
    - 说明癌症DNA突变检测有助于医生做出更准确的医疗决策，并在疗效和副作用间取得平衡。

11. **生物检体类型**
    - 介绍在进行DNA突变检测之前，必须先收集检体（如腫瘤组织或血液）。

12. **突变热点检测**
    - 描述基因体检测中的突变热点检测，说明这种方法可能遗漏选定基因之外的DNA突变。

13. **全方位癌症基因检测（CGP）**
    - 介绍全方位癌症基因检测的优势，包括能够检测更广泛的基因区段，提供更全面的突变信息。

14. **全方位癌症基因检测（CGP）**（续）
    - 详细描述全方位癌症基因检测的过程，从检体收集到数据分析和医生报告。

15. **癌症治疗方案**
    - 介绍癌症治疗通常根据肿瘤原发位置，并说明手术、放射线治疗和化学治疗的方法。

16. **癌症治疗方案**
    - 讲解新兴的标靶治疗与传统非标靶治疗（如化学治疗）

In [29]:
print(pdf_text)

Confidential –Strictly for Internal Use
CGP Patient Infographics
M-TW-0000141801什麼是癌症？
人體器官由組織構成，組織內含許多細胞。細胞是我們體內最小的功
能性單元。每個細胞都內含一個細胞核。
每個細胞核裡都有一組基因體。基因體內含許多基因。
基因由 DNA組成，內含所有人體運作的必要資訊。
器官組織細胞細胞核 基因體 染色體 DNA基因02什麼是癌症？
細胞的生命週期有限，因此會持續汰舊換新。
汰換的方式為生長和分裂。
過程中也會複製細胞內的 DNA，因此每個新的細胞都會有一組複製的
DNA。
細胞生長 細胞分裂03什麼是癌症？
DNA複製時可能會發生錯誤。
這些錯誤在醫學上被稱為「突變」。
正常的 DNA 變異的 DNA04什麼是癌症？
突變可能是由外在因子所造成 。
外在因子例如：抽煙、喝酒、化學 /輻射物質曝露。細菌或病毒（如：
人類乳突病毒 HPV）感染也可能造成突變。
突變也可能來自遺傳 ，這類突變稱為「 先天遺傳變異 」。
老化 生活習慣 化學物質 感染
遺傳輻射
正常的 DNA DNA突變後天
外在因子暴露
先天
遺傳變異05什麼是癌症？
有些突變可能會影響到細胞的正常運作和生長 。
隨著時間過去，沒有被修復的突變會逐漸累積在細胞裡，使細胞成為
生長 /分裂失控的癌細胞，最終形成惡性腫瘤 。
腫瘤內的癌細胞一旦進入血管或淋巴系統，將會在體內擴散，並在其
它器官形成腫瘤，此即所謂「癌症轉移」。
健康的細胞
正常的 DNA 變異的 DNA 癌細胞細胞
生長失控
惡性腫瘤
癌細胞進入血管或淋巴系統癌細胞在其它
器官形成腫瘤
轉移06癌症的命名
我們通常會依原發腫瘤位置命名癌症（如：乳癌、肝癌、肺癌）。
乳癌 肝癌 肺癌07化學治療
傳統上，癌症的療法取決於原發腫瘤位置。
過去最常用的是化學治療（以有毒的藥物攻擊快速生長的癌細胞）。
化學治療隨著醫學的進步，我們現在對癌症的 DNA突變有了更多的瞭解 。
舉例來說，雖然蘿拉和安娜得的都是乳癌，但她們有不同的 DNA突變，
因此可能需要接受不同的治療 。
相對地，不同位置的原發腫瘤也可能有相同的 DNA突變。因此安娜和詹
姆士或許可以接受相似的治療。08每位癌症病人都是獨特的
蘿拉 安娜 詹姆士突變A 突變B
乳癌 肺癌09每位癌症

In [30]:
# ! pip install openai PyPDF2 python-docx

   ---------------------------------------- 0.0/244.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/244.3 kB ? eta -:--:--
   ------ -------------------------------- 41.0/244.3 kB 326.8 kB/s eta 0:00:01
   ----------- --------------------------- 71.7/244.3 kB 435.7 kB/s eta 0:00:01
   ----------- --------------------------- 71.7/244.3 kB 435.7 kB/s eta 0:00:01
   ------------------------------- ------ 204.8/244.3 kB 827.9 kB/s eta 0:00:01
   ------------------------------- ------ 204.8/244.3 kB 827.9 kB/s eta 0:00:01
   -------------------------------------- 244.3/244.3 kB 680.5 kB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
import openai
import os
from PyPDF2 import PdfReader
import docx
import configparser
import time

# Set up OpenAI API key
config = configparser.ConfigParser()
config.read('config.ini')
OPENAI_API_KEY = config['DEFAULT']['OPENAI_API_KEY']
client = openai.OpenAI(api_key=OPENAI_API_KEY)

def read_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

def read_docx(file_path):
    doc = docx.Document(file_path)
    text = ""
    for para in doc.paragraphs:
        text += para.text + "\n"
    return text

def read_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def upload_file(file_path):
    with open(file_path, "rb") as file:
        return client.files.create(file=file, purpose="fine-tune")  # Changed purpose to "fine-tune"

def create_assistant(name, instructions):
    assistant = client.beta.assistants.create(
        name=name,
        instructions=instructions,
        model="gpt-4-1106-preview",
        tools=[{"type": "file_search"}]  # Changed to "file_search"
    )
    return assistant

def create_thread():
    return client.beta.threads.create()

def add_message_to_thread(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content
    )

def run_assistant(thread_id, assistant_id):
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )
    return run

def get_response(thread_id, run_id):
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        if run.status == "completed":
            break
        time.sleep(1)  # Wait for 1 second before checking again
    
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    return messages.data[0].content[0].text.value

# Main execution
if __name__ == "__main__":
    # Create an assistant
    assistant = create_assistant(
        "RAG Assistant",
        "You are a helpful assistant that answers questions based on the provided documents."
    )
    
    # Upload files
    pdf_file = upload_file("data/基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf")
    
    # Create a thread
    thread = create_thread()
    
    # Add a message to the thread
    add_message_to_thread(thread.id, "What are the key points discussed in the uploaded documents?")
    
    # Run the assistant
    run = run_assistant(thread.id, assistant.id)
    
    # Get the response
    response = get_response(thread.id, run.id)
    print("Assistant's response:", response)


Assistant's response: It appears that there are no documents uploaded for me to search through. Please upload the document(s) you would like me to review, and then I can help identify the key points discussed in them.


In [49]:
import openai
import os
from PyPDF2 import PdfReader
import docx
import configparser
import time

# Set up OpenAI API key
config = configparser.ConfigParser()
config.read('config.ini')
OPENAI_API_KEY = config['DEFAULT']['OPENAI_API_KEY']
openai.api_key = OPENAI_API_KEY

def read_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

def read_docx(file_path):
    doc = docx.Document(file_path)
    text = ""
    for para in doc.paragraphs:
        text += para.text + "\n"
    return text

def read_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def upload_file(file_path):
    with open(file_path, "rb") as file:
        response = openai.files.create(file=file, purpose="fine-tune")
    return response

def create_assistant(name, instructions):
    assistant = openai.beta.assistants.create(
        name=name,
        instructions=instructions,
        model="gpt-4-1106-preview",
        tools=[{"type": "file_search"}]
    )
    return assistant

def create_thread():
    return openai.beta.threads.create()

def add_message_to_thread(thread_id, content):
    return openai.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content
    )

def run_assistant(thread_id, assistant_id):
    run = openai.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )
    return run

def get_response(thread_id, run_id):
    while True:
        run = openai.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        if run.status == "completed":
            break
        time.sleep(1)
    
    messages = openai.beta.threads.messages.list(thread_id=thread_id)
    return messages.data[0].content[0].text.value

# Main execution
if __name__ == "__main__":
    # Create an assistant
    assistant = create_assistant(
        "RAG Assistant",
        "You are a helpful assistant that answers questions based on the provided documents."
    )
    
    # Upload files
    pdf_file = upload_file("data/基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf")
    print(f"Uploaded file ID: {pdf_file.id}")
    
    # Create a thread
    thread = create_thread()
    
    # Add a message to the thread
    add_message_to_thread(thread.id, "What are the key points discussed in the uploaded documents?")
    
    # Run the assistant
    run = run_assistant(thread.id, assistant.id)
    
    # Get the response
    response = get_response(thread.id, run.id)
    print("Assistant's response:", response)


Uploaded file ID: file-d2j7UvsXB85dDB9dXQxnGTP6
Assistant's response: It seems there are no uploaded documents to reference for your question. Please upload a document you'd like me to analyze, and I'll be happy to help you identify the key points discussed in it.


In [51]:
# Create an assistant
assistant = create_assistant(
    "RAG Assistant",
    "You are a helpful assistant that answers questions based on the provided documents."
)

# Upload files
pdf_file = upload_file("data/基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf")
print(f"Uploaded file ID: {pdf_file.id}")

# Create a thread
thread = create_thread()

# Add a message to the thread
add_message_to_thread(thread.id, "What are the key points discussed in the uploaded documents?")

# Run the assistant
run = run_assistant(thread.id, assistant.id)

# Get the response
response = get_response(thread.id, run.id)
print("Assistant's response:", response)


Uploaded file ID: file-hhi3LXZ9Ip0ir0d63jclJaQJ
Assistant's response: It appears that there are no uploaded documents to reference. If you have documents that you would like to discuss, please upload them, and I can help identify key points within those documents.


# 官網範例

In [54]:
from openai import OpenAI
# import openai
import os
from PyPDF2 import PdfReader
import docx
import configparser
import time

# Set up OpenAI API key
config = configparser.ConfigParser()
config.read('config.ini')
OPENAI_API_KEY = config['DEFAULT']['OPENAI_API_KEY']
client.api_key = OPENAI_API_KEY
# client = OpenAI()
 
assistant = client.beta.assistants.create(
  name="NGS Expert Assistant",
  instructions="You are an NGS expert. Use your knowledge base to answer questions about NGS.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

In [55]:
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="Financial Statements")
 
# Ready the files for upload to OpenAI
# file_paths = ["edgar/goog-10k.pdf", "edgar/brka-10k.txt"]
file_paths = ["data/基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [56]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [57]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open("data/基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf", "rb"), purpose="assistants"
)
 
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "這個文檔在說什麼?",
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)
 
# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_KEsnkCge9rP2C6TxnCQ19ViM'])


In [58]:
from typing_extensions import override
from openai import AssistantEventHandler, OpenAI
 
# client = OpenAI()
 
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))


# Then, we use the stream SDK helper
# with the EventHandler class to create the Run
# and stream the response.

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account.",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()


assistant > file_search


assistant > 這個文檔主要講述了癌症及其基因檢測的相關知識，包括癌症的定義、癌細胞的特性、導致癌症的突變原因、以及不同類型的基因檢測方法和其臨床應用。具體內容涵蓋以下幾個主要方面:

1. **癌症及其成因**：
    - 人體器官由組織構成，組織內含許多細胞。細胞內的DNA 出現異常突變可能導致癌症[0][1]。
    - 突變可以由外在因子（如吸煙、飲酒等）或遺傳因素引起[2][3]。

2. **基因檢測和癌症治療**：
    - 基因檢測可以幫助識別癌細胞中的特定DNA突變，從而做出更精準的醫療決策。例如，全方位癌症基因檢測（CGP）能夠檢測多個癌症相關的DNA突變，提供更完整的基因譜[4][5]。
    - 基因檢測結果可以有助於制定個性化的治療方案，包括標靶治療、免疫療法和參加臨床試驗[6][7]。

3. **癌症治療方案**：
    - 傳統的治療方法包括手術、化學治療和放射治療[8][9]。
    - 新的治療方法如標靶治療和免疫治療，可以更加精準地攻擊癌細胞而不損傷健康細胞[10][11]。

總體而言，文檔介紹了癌細胞的形成過程、基因突變的原因及其檢測方法，以及不同的癌症治療策略，重點強調了精準醫療的重要性。
[0] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf
[1] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf
[2] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf
[3] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf
[4] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf
[5] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf
[6] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.p

In [2]:
!pip install python-dotenv


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import openai
import os
import time
from dotenv import load_dotenv
from typing_extensions import override
from openai import AssistantEventHandler

# 載入 .env 文件中的環境變量
load_dotenv()

# 從環境變量中獲取 OpenAI API 金鑰
openai.api_key = os.getenv("OPENAI_API_KEY")

# 創建 OpenAI 客戶端
client = openai.OpenAI()

# 創建新的向量儲存
def create_vector_store():
    try:
        vector_store = client.beta.vector_stores.create(name="基因檢測知識庫")
        print(f"新向量儲存創建成功，ID: {vector_store.id}")
        return vector_store
    except Exception as e:
        print(f"創建向量儲存時發生錯誤: {e}")
        raise

# 上傳文件到向量儲存
def upload_files_to_vector_store(vector_store, file_paths):
    try:
        file_streams = [open(path, "rb") for path in file_paths]
        file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
            vector_store_id=vector_store.id,
            files=file_streams
        )
        print(f"文件批次狀態: {file_batch.status}")
        print(f"文件數量: {file_batch.file_counts}")
        return file_batch
    except Exception as e:
        print(f"上傳文件到向量儲存時發生錯誤: {e}")
        raise

# 創建助手
def create_assistant(vector_store_id):
    try:
        assistant = client.beta.assistants.create(
            name="基因檢測專家助手",
            instructions="您是一位專業的基因檢測專家。使用您的知識庫來回答關於基因檢測的問題。",
            model="gpt-3.5-turbo",
            tools=[{"type": "file_search"}],
            tool_resources={"file_search": {"vector_store_ids": [vector_store_id]}}
        )
        print(f"助手創建成功，ID: {assistant.id}")
        return assistant
    except Exception as e:
        print(f"創建助手時發生錯誤: {e}")
        raise

class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\n助手 > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\n助手 > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))

def ask_assistant(assistant, thread, question):
    try:
        # 添加用戶消息到線程
        client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=question
        )
        print("用戶問題已添加到線程")

        # 運行助手並使用 EventHandler 來處理輸出
        with client.beta.threads.runs.stream(
            thread_id=thread.id,
            assistant_id=assistant.id,
            event_handler=EventHandler(),
        ) as stream:
            stream.until_done()

    except Exception as e:
        print(f"處理問題時發生錯誤: {e}")
        return f"發生錯誤: {str(e)}"

# 測試函數
def test_rag_system(assistant, thread):
    print("RAG 系統測試開始")
    print("請輸入您的問題，或輸入 'quit' 退出")
    
    while True:
        question = input("您的問題: ")
        if question.lower() == 'quit':
            break
        
        ask_assistant(assistant, thread, question)
    
    print("測試結束")

# 主程序
def main():
    try:
        vector_store = create_vector_store()
        file_paths = ["data/基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf", "data/基因檢測服務流程_檢測結果對癌症治療之效益_Patient brochure(A4)_M-TW-00001452.pdf"]
        file_batch = upload_files_to_vector_store(vector_store, file_paths)
        assistant = create_assistant(vector_store.id)
        thread = client.beta.threads.create()
        print(f"新線程創建，ID: {thread.id}")
        test_rag_system(assistant, thread)
    except Exception as e:
        print(f"程序執行過程中發生錯誤: {e}")

# 運行主程序
if __name__ == "__main__":
    main()

新向量儲存創建成功，ID: vs_a51LNSFoGvoaXTJPmJRhZfi2
文件批次狀態: completed
文件數量: FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)
助手創建成功，ID: asst_TSPloFydw7xkValxJSfCdAMz
新線程創建，ID: thread_uXqFkgkPTfVa08H9TIDpJZ85
RAG 系統測試開始
請輸入您的問題，或輸入 'quit' 退出


您的問題:  請說明你有哪些知識


用戶問題已添加到線程

助手 > file_search


助手 > 根據上傳的文件，基因檢測可以通過基因檢測服務流程進行，其中包括全方位癌症基因檢測（CGP）。CGP是一種全面的基因檢測方法，能夠廣泛檢測多個癌症相關的DNA突變，提供更完整的基因檢測藍圖，有助於提升醫療的精準度。此外，基因檢測也可以協助醫師制定更精確的癌症治療方案，例如手術切除、放射線治療、化學治療，甚至標靶治療和免疫療法等不同形式的治療選擇[0][1]。

藉由基因檢測，醫師可以更好地了解每位癌症患者的獨特情況，因為不同的患者可能具有不同的基因突變，因此需要量身定制的治療計畫。精準的治療決策需要在療效和副作用之間取得平衡，因此基因檢測提供了寶貴的資訊，有助於設計個性化的治療方案[2][3]。

整體而言，基因檢測在癌症治療中扮演重要角色，幫助醫師更好地理解患者的基因異常，從而制定更有效的治療策略，提高治療效果和患者的生存率。
[0] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf
[1] 基因檢測服務流程_檢測結果對癌症治療之效益_Patient brochure(A4)_M-TW-00001452.pdf
[2] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf
[3] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf


您的問題:  請整理"基因檢測服務流程_檢測結果對癌症治療之效益_Patient brochure(A4)_M-TW-00001452.pdf"每一頁的知識


用戶問題已添加到線程

助手 > file_search


助手 > 以下是"基因檢測服務流程_檢測結果對癌症治療之效益"手冊中每一頁所提供的知識摘要：

- 追蹤與監控可以增加治療效益，幫助篩選適合的治療介入[0]。
- 全方位癌症基因檢測能夠做更廣泛的基因區段檢測，提供完整的基因檢測藍圖，增加發現重要突變的機會，進而提高醫療的精準度[0]。
- 突變熱點檢測是基因體檢測的一種方法，它在選定的基因區段偵測特定的突變，然而使用此方法可能會遺漏非選定基因區段的DNA突變[0]。
- 每位癌症患者都是獨特的，具有不同的突變基因和數目，因此需要量身定制的治療方案[3]。
- 癌症治療方案可能包括手術切除、放射線治療、化學治療、標靶治療和免疫療法等不同形式的治療選擇，取決於腫瘤的原發位置和擴散情況[4]。

這些信息提供了關於基因檢測如何改善癌症治療效益的洞察。
[0] 基因檢測服務流程_檢測結果對癌症治療之效益_Patient brochure(A4)_M-TW-00001452.pdf
[1] 基因檢測服務流程_檢測結果對癌症治療之效益_Patient brochure(A4)_M-TW-00001452.pdf
[2] 基因檢測服務流程_檢測結果對癌症治療之效益_Patient brochure(A4)_M-TW-00001452.pdf
[3] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf
[4] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf


...

用戶問題已添加到線程

助手 > file_search


助手 > 這份文件共包含21頁，以下是每一頁的知識摘要：

1. 第1頁至第5頁介紹了癌症形成的過程，包括細胞汰換、DNA複製中的錯誤、突變可能的來源等[0]。

2. 第13頁至第15頁講解了全方位癌症基因檢測（CGP）的重要性，以及如何從腫瘤組織或血液檢體進行基因體檢測，以提高精準度[0]。

3. 第20至21頁討論了腫瘤突變負荷（TMB）的重要性，及如何根據基因檢測結果與醫師討論適合的治療方案[0]。

4. 其他頁面提到了每位癌症患者的獨特性，基因檢測如何幫助制定個性化治療方案，以及突變熱點檢測等基因檢測相關的知識[0]。

這些摘要提供了基因檢測在癌症治療中的重要性以及其應用的基本概念。
[0] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf
[1] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf
[2] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf
[3] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf


您的問題:  那16到19頁呢?


用戶問題已添加到線程

助手 > file_search


助手 > 以下是第16至19頁的知識摘要：

- 第16頁介紹了癌症治療方案中的新興標靶治療以及非標靶治療的差異，標靶治療能夠鎖定特定DNA突變的癌細胞，提高治療精準度；而非標靶治療可能同時傷害到健康細胞[0]。

- 第17頁介紹了免疫療法的作用方式，包括使癌細胞停止生長或減緩生長、阻止癌細胞擴散、協助免疫系統對抗癌細胞等[1]。

- 第18頁進一步探討了免疫檢查哨抑制劑如何作用，阻斷癌細胞的免疫檢查哨訊息，使免疫系統能重新辨認、攻擊癌細胞[2]。

- 第19頁討論了腫瘤突變負荷（TMB）的概念，指出TMB高的腫瘤較容易表現異常蛋白質，有助於免疫系統對抗癌細胞，全方位癌症基因檢測可以提供腫瘤的TMB程度資訊[2]。

這些知識摘要涵蓋了癌症治療中新興的治療方式以及基因檢測在評估治療方案中的重要性。
[0] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf
[1] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf
[2] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf
[3] 基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf


您的問題:  quit


測試結束


請說明你有哪些知識
請整理"基因檢測服務流程_檢測結果對癌症治療之效益_Patient brochure(A4)_M-TW-00001452.pdf"每一頁的知識
請整理"基因檢測類型與說明_CGP Patient Infographics-Slide M-TW-00001418.pdf"每一頁的知識